In [1]:
import spikeinterface.core as si
import spikeinterface.preprocessing as spre
import spikeinterface.widgets as sw
from probeinterface.plotting import plot_probe
import os
import pickle
import matplotlib.pyplot as plt

import SI_tools


###################################################
# Some default parameters for multiprocessing
###################################################
n_cpus = os.cpu_count()
n_jobs = n_cpus - 2

job_kwargs = dict(chunk_duration="1s", n_jobs=n_jobs, progress_bar=True)

######################################################################################################
# PARAMETER FOR "LOCAL REFERENCING"
exclude_radius_chans = 1 # Number of neighbor channels to exclude because are too close to the reference channel
include_radius_chans = 4 # Number of neighbor channels delineates the outer boundary of the annulus whose role is to exclude channels that are too far away

############################################################################################################################################################################################################
#                                                          Set the Parent Directories
############################################################################################################################################################################################################

#########################################################################################################################
# "parentRecordingFolder"
#   Folder from where to read NWB (read from server, unless you copied the files into the local machine)
parentRecordingFolder = 'Y:\Data_Albus\Albus_NeuralData\Albus-S1'

#########################################################################################################################
# "savePreprocessing"
#   If True, it will save PLOTS from PREPROCESSING STEPS in the "parentPreproFolder". 
#   If "localProcess_NWB" = False, then electrodeInformation & recording will be saved too (lazy object)
#   If False, this pipeline works mainly for exploration of Noise to run full preprocessing pipeline
savePreprocessing = False

# "parentPreproFolder":
#   Folder to save all the Plots & recording objects from the preprocessing (server is recommended to keep record of the sorting process)
parentPreproFolder = 'Y:\Data_Albus\Albus_NeuralData\Albus-S1-Prepro' 

######################################################################################################
# "localProcess_NWB" : 
#       if TRUE, it will copy NWB files into a temporary directory in the same disk as this script and it will deleted the NWB files at the end
#                it will not create recording objects due to the lost of path links after deleting the temprary files
#       if FALSE it will read files from original location and it will create recording objects with paths linked to the original NWB locations.
localProcess_NWB = False

In [2]:
######################################################################################################
# Set the Recording information 
######################################################################################################
sessionYear = 2024
sessionMonth = 4
sessionDay = 1

In [ ]:
######################################################################################################
# GET INFORMATION ABOUT ELECTRODE GROUPS AND SESSIONS PER GROUP
# If you know the electrodeGroupName and the sessions you want to inspect, you can skip this cell
######################################################################################################
    
SI_tools.print_recordingInfo(
    parentRecordingFolder = parentRecordingFolder, 
    sessionYear = sessionYear, 
    sessionMonth = sessionMonth, 
    sessionDay = sessionDay
    )


In [ ]:
#########################################################################################################################
# ElectrodeGroupe Name (based on NWBfile naming schema) 
# Note: if there were different coordinates within the same day, you need to add '-sub1', '-sub2', etc. 
# It is recommended to run "SI_run_prepro" first, it will create folder for all the unique electrode groups from that day.
# In case you want to explore how many electrodeGroup-subX exists, run the above cell:
# SI_tools.print_recordingInfo(parentRecordingFolder, sessionYear, sessionMonth, sessionDay)
###########################################################################################################################
electrodeGroup_Name = 'raw-C1-257' 

#########################################################################################################################
# Session index is based on the dateTime. Zero is the first session of the day from the selected electrodeGroup
# If None, it will use all sessions from this electrodeGroup in this experiment Day
session_index = [0, 1] 

#######################################################################################################################
# Get Recording Object:
si_recording, electrodeGroup_sessions = SI_tools.get_si_recording(
                    parentRecordingFolder = parentRecordingFolder,
                    sessionYear = sessionYear, 
                    sessionMonth = sessionMonth, 
                    sessionDay = sessionDay, 
                    electrodeGroup_Name = electrodeGroup_Name, 
                    session_index = session_index,
                    localProcess_NWB = localProcess_NWB
                )


# GET CHANNEL-CONTACT SPACING ("y" coordinate)
step_chan = si_recording.get_annotation('y_contacts_distance_um') 


######################################################################################################
#       If save Preprocessing is TRUE, CREATE Folders to save Figures and objects
######################################################################################################
if savePreprocessing and not localProcess_NWB:

    #######################################################################################################################
    # Get/Create FOLDER paths to save Figures
    #######################################################################################################################
    # Create General Session Folder to save Figures
    sessionFolder = os.path.join(os.path.abspath(parentPreproFolder), 'exp{}-{:02d}-{:02d}_sorting'.format(sessionYear, sessionMonth, sessionDay))
    if not os.path.isdir(sessionFolder):
        os.makedirs(sessionFolder)
    
    #############################################################################
    # Create Folder to save ElectrodeGroup-Session Preprocessing
    elecGroupSessName = '{}-{}'.format(electrodeGroup_sessions['sessID'], electrodeGroup_sessions['electrodeName'])
    elecGroupSessFolder = os.path.join(sessionFolder, elecGroupSessName)
    if not os.path.isdir(elecGroupSessFolder):
        os.mkdir(elecGroupSessFolder)

    ###########################################################
    # Save ElectrodeGroupInformation
        pickle.dump(electrodeGroup_sessions, open(os.path.join(elecGroupSessFolder, elecGroupSessName + '_electrodeGroupInfo.pkl'), 'wb' ))


In [ ]:
######################################################################################################
# Print full Info (properties and annotations) of the file 
######################################################################################################

# Properties:
print('Property keyNames: \n', si_recording.get_property_keys(), '\n')
print('Main properties: \n', si_recording._main_properties, '\n')
print('ALL properties: \n', si_recording._properties, '\n')
print('example how to get a property : \n', si_recording.get_property('brain_area'), '\n')

# Annotations
print('Annotations keyNames: \n', si_recording.get_annotation_keys(), '\n')
print('Main annotations keyNames: \n', si_recording._main_annotations, '\n')
print('ALL annotations: \n', si_recording._annotations, '\n')
print('example how to get an annotation: \n', si_recording.get_annotation('is_filtered'), '\n')



In [ ]:
######################################################################################################
#                                      PLOT THE PROBE
######################################################################################################
%matplotlib ipympl
if si_recording.has_probe():
    plot_probe(probe=si_recording.get_probe(), with_contact_id=False, with_device_index=True)

In [7]:
######################################################################################################
#                              Create filtered and CMR (common-median-reference)
######################################################################################################

##########################################################
# High-pass filter
si_recording_filter_pre = spre.bandpass_filter(recording=si_recording, freq_min=500) 

##########################################################
# Median zero-center signal (per channel basis)
si_recording_center_pre = spre.center(recording = si_recording_filter_pre, mode='median')

##########################################################
# Apply Common Median Reference
##########################################################
if si_recording.get_num_channels()>1:
    si_recording_cmr_pre = spre.common_reference(recording=si_recording_center_pre, reference='local', operator='median', local_radius=(exclude_radius_chans * step_chan, include_radius_chans * step_chan ))
else:
    si_recording_cmr_pre = si_recording_center_pre


In [ ]:
###################################################################################################################
#                           If more than one session was selected
#               plot epochs around the concatenation to inspect possible artifacts
###################################################################################################################

%matplotlib ipympl

SI_tools.plot_concatenations(si_recording_dict={'Raw': si_recording, 'Filter': si_recording_filter_pre, 'CMR': si_recording_cmr_pre}, plot_windows_secs=0.005)

In [ ]:
######################################################################################################
#                              Explore the ENTIRE raw, filtered, CMR
######################################################################################################

##########################################################
# PLOT Traces
%matplotlib ipympl 

sw.plot_traces({'Raw': si_recording, 'Filter': si_recording_filter_pre, 'CMR': si_recording_cmr_pre}, 
    segment_index=0,
    time_range=[0, 30], 
    backend="ipywidgets"
    )

In [ ]:
#####################################################################
#                           DENOISE
#####################################################################
# TO DO:
# Remove artifact due to IntraCorticalMicroStimulation (ICMS)

In [ ]:
#####################################################################
#                           DENOISE
#####################################################################

#####################################################################
# PLOT Power Spectrum Density 
# if compare_CMR=True,  it will apply Global CommonMedianReference to compare

%matplotlib ipympl
SI_tools.plotPSD_randomChunks(si_recording_filter_pre, compare_CMR=True, plot_by_channel=False, chan_radius=(exclude_radius_chans, include_radius_chans))


In [ ]:
#####################################################################
#                           DENOISE (PLOT BY CHANNEL)
#####################################################################

################################################################################################
# In case there is need to further inspect by channel, run it with "plot_by_chanel=True"

%matplotlib ipympl
SI_tools.plotPSD_randomChunks(si_recording_filter_pre, compare_CMR=True, plot_by_channel=True, chan_radius=(exclude_radius_chans, include_radius_chans))

In [12]:
#####################################################################
#                           DENOISE
# If all channels were plot, it is recommended to clear the previous
# cell to avoid memory problems
#####################################################################

#########################################################################################################
# If there is noise at a specefic frequency, remove it with a notch filter
# If you didn't find any specific noise then set to "None" (it will just update Centered recording)
noisy_freq = None

# Example: Remove 2.6 KHz
# noisy_freq = 2600

################################################
# choose if you want to plot by channel or not
plot_by_channel = True

if noisy_freq is not None:

    si_recording_denoise = spre.notch_filter(si_recording, freq=noisy_freq, q=10)

    ##########################################################
    # High-pass filter
    si_recording_filter = spre.bandpass_filter(recording=si_recording_denoise, freq_min=500) 

    ##########################################################
    # Median zero-center signal (per channel basis)
    si_recording_center = spre.center(recording=si_recording_filter, mode='median')

    #####################################################################
    # PLOT again Power Spectrum Density 
    %matplotlib ipympl
    SI_tools.plotPSD_randomChunks(si_recording_filter, compare_CMR=True, plot_by_channel=plot_by_channel, chan_radius=(exclude_radius_chans, include_radius_chans))

else:
    
    si_recording_center = si_recording_center_pre

In [ ]:
#############################################################################################################
#                       Detect bad channels
#############################################################################################################

print('Detecting Bad Channels ....... \n')
# Use recObj centered (before CMR)
bad_channel_ids, channel_labels = spre.detect_bad_channels(recording=si_recording_center, method="coherence+psd")

# add channel_labels 'coeherence+psd' : good/dead/noise/out 'std, mad, neighborhood_r2' : good/noise
si_recording_center.set_property(key='channel_labels', values=channel_labels)

##########################################################
# Remove bad channels 
si_recording_center_cleanChans = si_recording_center.remove_channels(remove_channel_ids=bad_channel_ids)

######################################################################################################
# PLOT THE PROBE & channel labels

%matplotlib ipympl
if si_recording.has_probe():

    fig, ax = plt.subplots(ncols=2, nrows=1, figsize=(12, 8))

    plot_probe(probe=si_recording.get_probe(), ax = ax[0], with_contact_id=False, with_device_index=True)

    y_probe = si_recording.get_channel_locations(axes='y').flatten()
    ch_indx = si_recording.ids_to_indices()
    contact_ids = si_recording.get_channel_ids()
    for ch in ch_indx:
        ax[1].plot(0, y_probe[ch], marker="s", markeredgecolor=(0, 0, 1, 1), markerfacecolor=(0, 0, 1, 0.5), markeredgewidth = 1, markersize=6)
        ax[1].text(-0.5, y_probe[ch], 'ch={}'.format(contact_ids[ch]), horizontalalignment='left', color=(0, 0, 0, 1), fontsize=8)
        if channel_labels[ch]=='good':
            colorT = (0, 0, 1, 1)
            fweight = 'normal'
        else:
            colorT = (1, 0, 0, 1)
            fweight = 'bold'
        ax[1].text(0.5, y_probe[ch], channel_labels[ch], horizontalalignment='right', color=colorT, fontsize=8, fontweight=fweight)
    ax[1].set_xlim(-1, 1)

else:
    print('Channel labels : \n\t', channel_labels)
    print('\n{} bad channels detected\n\tBad channel Index: {}'.format(len(bad_channel_ids), bad_channel_ids))


In [ ]:
######################################################################################################
#                            Apply LOCAL Common Median Reference
######################################################################################################

##########################################################
if si_recording_center_cleanChans.get_num_channels()>1:
    si_recording_cmr = spre.common_reference(recording=si_recording_center_cleanChans, reference='local', operator='median', local_radius=(exclude_radius_chans * step_chan, include_radius_chans * step_chan ))
else:
    si_recording_cmr = si_recording_center_cleanChans


###################################################
# Add some annotations:
si_recording_cmr.annotate(is_filtered=True)
si_recording_cmr.annotate(is_centered=True)
si_recording_cmr.annotate(centered_mode = 'median')
si_recording_cmr.annotate(is_referenced=True)    
si_recording_cmr.annotate(reference = 'local')
si_recording_cmr.annotate(reference_mode = 'median')

print('Referenced properties:\n', si_recording_cmr._properties, '\n')
print('Referenced annotations:\n', si_recording_cmr._annotations, '\n')

In [ ]:
######################################################################################################
#                            Explore CMR before and after
######################################################################################################
# Match channels across recording objects
chans_match = list( set(si_recording_cmr_pre.channel_ids) &  set(si_recording_cmr.channel_ids) )
chans_match_ordered = [i for i in si_recording_cmr_pre.channel_ids if i in chans_match]

si_recording_cmr_pre_match = si_recording_cmr_pre.channel_slice(channel_ids=chans_match_ordered)
si_recording_cmr_match = si_recording_cmr.channel_slice(channel_ids=chans_match_ordered)

%matplotlib ipympl
sw.plot_traces({'CMR_orig': si_recording_cmr_pre_match,'CMR_clean': si_recording_cmr_match}, 
    segment_index=0,
    time_range=[0, 10], 
    backend="ipywidgets"
    )

In [17]:
#############################################################################################################
# If everything looks fine & NWB files were not move from original location: 
# Save Recording Object (LAZY without traces)
#############################################################################################################

if savePreprocessing and not localProcess_NWB:

    si_recording_cmr.dump_to_pickle(os.path.join(elecGroupSessFolder, elecGroupSessName  + '_SIrecording'))


    ######################################################################################################
    # Test loading the PREPRO file and get some information to confirm everything is ok
    si_recording_cmr_loaded = si.load_extractor(os.path.join(elecGroupSessFolder, elecGroupSessName  + '_SIrecording.pkl')) 

    print(si_recording_cmr_loaded, '\n')

    # Properties
    print('Property keyNames: \n', si_recording_cmr_loaded.get_property_keys(), '\n')
    print('Main properties: \n', si_recording_cmr_loaded._main_properties, '\n')
    print('ALL properties: \n', si_recording_cmr_loaded._properties, '\n')

    # Annotations
    print('Annotations keyNames: \n', si_recording_cmr_loaded.get_annotation_keys(), '\n')
    print('Main annotations keyNames: \n', si_recording_cmr_loaded._main_annotations, '\n')
    print('ALL annotations keyNames: \n', si_recording_cmr_loaded._annotations, '\n')
